<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Repetition codes on real devices

* Requires qiskit-ignis 0.2.0 or greater.

### Contributors

James R. Wootton, IBM Research

In [another notebook](../../qiskit/ignis/repetition_code.ipynb) we saw how to use the `RepetitionCode` and `GraphDecoder` tools from Ignis. Here we'll specifcially look at how to use them with real hardware.

In [1]:
from qiskit.ignis.verification.topological_codes import RepetitionCode
from qiskit.ignis.verification.topological_codes import GraphDecoder
from qiskit.ignis.verification.topological_codes import lookuptable_decoding, postselection_decoding

Let's start off easy, with a `d=3`, `T=1` repetition code.

In [2]:
d = 3
T = 1
code = RepetitionCode(d,T)

The circuits for this are found in `code.circuit['0']` (for an encoded `0`) and code.circuit['1'] (for an encoded `1`). We can use `count_ops` to see which operations these contain.

In [3]:
code.circuit['0'].count_ops()

{'cx': 4, 'measure': 5, 'reset': 2, 'barrier': 1}

In [4]:
code.circuit['1'].count_ops()

{'x': 3, 'barrier': 2, 'cx': 4, 'measure': 5, 'reset': 2}

Both contain 4 `cx` gates, which is exactly what we'd expect from this code.

Now let's set up a real device as a backend, and compile these circuits for it.

In [5]:
from qiskit import IBMQ
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager

IBMQ.load_accounts()
backend = IBMQ.get_backend('ibmq_16_melbourne')
qc0 = transpile(code.circuit['0'], backend=backend)
qc1 = transpile(code.circuit['1'], backend=backend)

Let's see what has happened to the gates they contain.

In [6]:
print('gates for encoded 0 = ', qc0.count_ops())
print('gates for encoded 1 = ', qc1.count_ops())

gates for encoded 0 =  {'u2': 14, 'cx': 7, 'measure': 5, 'reset': 2, 'barrier': 1}
gates for encoded 1 =  {'u3': 3, 'barrier': 2, 'u2': 14, 'cx': 7, 'measure': 5, 'reset': 2}


The single qubit gates are now all `u2`s and `u3`s, as usually happens when we compile to real hardware. But the number of `cx` gates has increased! This implies some remapping has occurred, which is not what we want for error correction. To avoid this, we first need to provide look at the coupling map of the device.

In [7]:
coupling_map = backend.configuration().coupling_map
print(coupling_map)

[[1, 0], [1, 2], [2, 3], [4, 3], [4, 10], [5, 4], [5, 6], [5, 9], [6, 8], [7, 8], [9, 8], [9, 10], [11, 3], [11, 10], [11, 12], [12, 2], [13, 1], [13, 12]]


We can think of the qubits of a repetition code as sitting along a line (alternating between 'code' and 'link' qubits), with each qubit interacting only with its neighbours. So we need to look at the coupling map and find a line that covers as many qubits as we can. One possibility is

In [8]:
line = [0,1,2,3,4,5,6,8,9,10,11,12,13]

With this we can set up an `initial_layout` dictionary, which tells us exactly which qubit in the circuit corresponds to which qubits on the device.

In [9]:
def get_initial_layout(code,line):
    initial_layout = {}
    for j in range(d):
        initial_layout[(code.code_qubit,j)] = line[2*j]
    for j in range(d-1):
        initial_layout[(code.link_qubit,j)] = line[2*j+1]
    return initial_layout

initial_layout = get_initial_layout(code,line)

print(initial_layout)

{(QuantumRegister(3, 'code_qubit'), 0): 0, (QuantumRegister(3, 'code_qubit'), 1): 2, (QuantumRegister(3, 'code_qubit'), 2): 4, (QuantumRegister(2, 'link_qubit'), 0): 1, (QuantumRegister(2, 'link_qubit'), 1): 3}


Now let's compile using this.

In [10]:
qc0 = transpile(code.circuit['0'], backend=backend, initial_layout=initial_layout)
qc1 = transpile(code.circuit['1'], backend=backend, initial_layout=initial_layout)

print('gates for d = '+str(d)+' with encoded 0:', qc0.count_ops())
print('gates for d = '+str(d)+' with encoded 1:', qc1.count_ops())

gates for d = 3 with encoded 0: {'u2': 6, 'cx': 4, 'measure': 5, 'reset': 2, 'barrier': 1}
gates for d = 3 with encoded 1: {'u3': 3, 'barrier': 2, 'u2': 6, 'cx': 4, 'measure': 5, 'reset': 2}


The number of `cx` gates is now as it should be.

Before running, we need to remove the reset gates. For `T=1`, they don't actually do anything anyway. But since they aren't currently supported on hardware, they may cause use trouble.

In [11]:
def remove_reset(qc):
    qc.data = [ gate for gate in qc.data if gate[0].name!='reset' ]
    return qc
qc0 = remove_reset(qc0)
qc1 = remove_reset(qc1)

Now we can actually run the circuits. So let's set up a function to do this for us.

In [12]:
from qiskit import execute, Aer
from qiskit.providers.aer import noise

def get_syndrome(circuits,backend,sim=False,shots=8192):
    
    if sim:
        noise_model = noise.device.basic_device_noise_model(backend.properties())
        job = execute( circuits, Aer.get_backend('qasm_simulator'), noise_model=noise_model, shots=shots )
    else:
        job = execute( circuits, backend, shots=shots )
    raw_results = {}
    for log in ['0','1']:
        raw_results[log] = job.result().get_counts(log)

    return code.process_results( raw_results )

This has a `sim` argument, with which we can choose whether to actually use the real device, or just use the noise model we get from the device in a simulation.

Let's just simulate for now.

In [13]:
sim = True

In [14]:
results = get_syndrome([qc0,qc1],backend,sim=sim)
print(results)

{'0': {'1 1  10 01': 8, '1 0  11 01': 20, '0 1  11 01': 8, '0 0  11 00': 137, '0 1  10 11': 67, '1 0  01 11': 61, '1 0  00 01': 35, '0 0  11 11': 177, '1 1  10 10': 1, '0 1  11 10': 23, '1 0  10 00': 159, '0 0  10 01': 99, '0 0  10 10': 1250, '0 0  00 11': 342, '0 1  01 00': 82, '0 0  00 00': 4180, '0 0  01 01': 559, '1 0  11 10': 20, '1 1  01 01': 1, '0 1  00 10': 17, '1 0  00 10': 427, '1 1  00 11': 13, '1 1  11 00': 2, '0 1  01 11': 16, '1 0  01 00': 21, '1 0  10 11': 16, '1 1  01 10': 2, '0 1  00 01': 175, '0 0  01 10': 269, '0 1  10 00': 5}, '1': {'1 1  10 01': 102, '1 0  11 01': 10, '0 1  11 01': 32, '0 0  11 00': 5, '1 1  00 00': 4099, '1 0  01 11': 20, '1 0  00 01': 203, '0 1  10 11': 13, '1 1  10 10': 1195, '1 1  11 11': 173, '0 1  11 10': 25, '1 0  10 00': 8, '0 0  10 01': 7, '0 0  00 11': 21, '0 1  01 00': 23, '0 0  00 00': 2, '1 0  11 10': 32, '1 1  01 01': 563, '0 1  00 10': 416, '1 0  00 10': 14, '1 1  00 11': 352, '1 1  11 00': 162, '0 1  01 11': 58, '1 0  01 00': 106, '

And we can decode the results.

In [15]:
dec = GraphDecoder(code)

logical_prob_match = dec.get_logical_prob(results)
logical_prob_lookup = lookuptable_decoding(results,results)
logical_prob_post = postselection_decoding(results)

for log in ['0','1']:
    print('d =',d,',log =',log)
    print('logical error probability for matching      =',logical_prob_match[log])
    print('logical error probability for lookup table  =',logical_prob_lookup[log])
    print('logical error probability for postselection =',logical_prob_post[log])
    print('')

d = 3 ,log = 0
logical error probability for matching      = 0.0322265625
logical error probability for lookup table  = 0.02294921875
logical error probability for postselection = 0.0

d = 3 ,log = 1
logical error probability for matching      = 0.034423828125
logical error probability for lookup table  = 0.0208740234375
logical error probability for postselection = 0.000487685930260912



Now let's see what happens when we look at different code sizes.

In [16]:
for d in range(3,8):
    
    code = RepetitionCode(d,1)
    
    initial_layout = get_initial_layout(code,line)
    circuits = [ transpile(code.circuit[log], backend=backend, initial_layout=initial_layout) for log in ['0','1'] ]
    circuits = [ remove_reset(qc) for qc in circuits ]
    
    print('gates for d = '+str(d)+' with encoded 0:', circuits[0].count_ops(), '\n')

    results = get_syndrome(circuits,backend,sim=sim)   
    
    dec = GraphDecoder(code)

    logical_prob_match = dec.get_logical_prob(results)
    logical_prob_lookup = lookuptable_decoding(results,results)
    logical_prob_post = postselection_decoding(results)

    for log in ['0','1']:
        print('d =',d,',log =',log)
        print('logical error probability for matching      =',logical_prob_match[log])
        print('logical error probability for lookup table  =',logical_prob_lookup[log])
        print('logical error probability for postselection =',logical_prob_post[log])
        print('')
    print('')

gates for d = 3 with encoded 0: {'u2': 6, 'cx': 4, 'measure': 5, 'barrier': 1} 

d = 3 ,log = 0
logical error probability for matching      = 0.0338134765625
logical error probability for lookup table  = 0.026123046875
logical error probability for postselection = 0.0

d = 3 ,log = 1
logical error probability for matching      = 0.035400390625
logical error probability for lookup table  = 0.02197265625
logical error probability for postselection = 0.00048204386599180526


gates for d = 4 with encoded 0: {'u2': 12, 'cx': 6, 'measure': 7, 'barrier': 1} 

d = 4 ,log = 0
logical error probability for matching      = 0.0203857421875
logical error probability for lookup table  = 0.0157470703125
logical error probability for postselection = 0.0

d = 4 ,log = 1
logical error probability for matching      = 0.0218505859375
logical error probability for lookup table  = 0.01318359375
logical error probability for postselection = 0.0


gates for d = 5 with encoded 0: {'u2': 12, 'cx': 8, 'measure':

If we look only at odd `d`, here we see a steady decrease in the logical error probability, as expected. A decrease is not seen between each odd $d$ and the following $d+1$ for the matching decoder. This is due to the fact that the number of errors required to overturn a clear majority is the same in both cases.

Note that all the jobs above could have been sent in a single batch, which speeds things up for real devices. Also, two separate sets of results for each code should really be obtained and used in `lookuptable_decoding` to prevent over-fitting. These things were not done to keep this a simpler and clearer tutorial. 

In [ ]:
keywords = {'Topics': ['Ignis', 'Quantum error correction'], 'Commands': ['`RepetitionCode`', '`GraphDecoder`', '`transpile`', '`initial_layout`']}